In [1]:
import numpy as np
import scipy.optimize as opt

OpenBLAS WARNING - could not determine the L2 cache size on this system, assuming 256k
OpenBLAS WARNING - could not determine the L2 cache size on this system, assuming 256k


# 3.1
次の関数の勾配とヘッセ行列を求めよ.

(i)
>$$f(x) = 2 x_1^3 - x_1^2x_2 + 2x_2^2.$$

$$
\nabla f(x) = \begin{bmatrix}
                6 x_1^2 - 2x_1x_2\\
                -x_1^2 + 4x_2
              \end{bmatrix},\ 
\nabla^2 f(x) = \begin{bmatrix}
                  12x_1 - 2x_2  &  -2x_1\\
                  -2x_1  & 4
                \end{bmatrix}.
$$

In [2]:
# 最適解なし
f = lambda x: 2*(x[0]**3) - (x[0]**2)*x[1] + 2*(x[1]**2)
x0 = np.array([1.0, 1.0])
res = opt.minimize(f, x0, method="nelder-mead", options={"disp": True})

In [3]:
# methodを指定しないと嘘解が求まっちゃう
res = opt.minimize(f, x0, options={"disp": True})
res.x

Optimization terminated successfully.
         Current function value: 0.000000
         Iterations: 14
         Function evaluations: 48
         Gradient evaluations: 16


array([ 8.47274610e-04, -7.46373369e-09])

(ii)
>$$ f(x) = (x_1 - 3)^2 + x_1 x_2 + \frac{1}{16}x_2^4 + (x_2 - 1)^2. $$

$$
    \nabla f(x) = \begin{bmatrix}
                    2x_1 + x_2 - 6 \\
                    x_1 + \frac{1}{4} x_2^3 + 2x_2 - 2
                  \end{bmatrix},\ 
    \nabla^2 f(x) = \begin{bmatrix}
                      2 & 1\\
                      1 & \frac{3}{4}x_2^2 + 2
                    \end{bmatrix}.
$$

In [4]:
f = lambda x: (x[0]-3)**2 + x[0]*x[1] + (1/16)*(x[1]**4) + (x[1]-1)**2
x0 = np.array([1.0, 1.0])
res = opt.minimize(f, x0, method="nelder-mead", options={"disp": True})
res.x

Optimization terminated successfully.
         Current function value: 0.677505
         Iterations: 46
         Function evaluations: 88


array([ 3.3128884 , -0.62579102])

In [5]:
# 勾配を指定して BFGS を使った方が収束が早い.
def jac(x):
    output = np.zeros(2)
    output[0] = 2*x[0] + x[1] - 6
    output[1] = x[0] + (1/4)*(x[1]**3) + 2*x[1] - 2
    return output

res = opt.minimize(f, x0, method="BFGS", jac=jac, options={"disp": True})
res.x

Optimization terminated successfully.
         Current function value: 0.677505
         Iterations: 7
         Function evaluations: 8
         Gradient evaluations: 8


array([ 3.31290812, -0.6258167 ])

(iii)
>$$ f(x) = \| Ax-b \|_2^2 + \gamma \|x\|_2^2. $$

$$
    \nabla f(x) = 2(A'A + \gamma I)x - 2A'b,\ 
    \nabla^2 f(x) = 2(A'A + \gamma I).
$$

(iv)
>$$ f(x) = \sum_{j=1}^n \log\left(1+\exp(-x_j)\right). $$

$$
    \nabla f(x) = \begin{bmatrix}
                    -\frac{\exp(-x_1)}{1 + \exp(-x_1)} \\
                    \vdots \\
                    -\frac{\exp(-x_n)}{1 + \exp(-x_n)}
                  \end{bmatrix},\ 
    \nabla^2 f(x) = \begin{bmatrix}
                      \frac{\exp(-x_1)}{(1 + \exp(-x_1))^2} & & O\\
                      & \ddots & \\
                      O & & \frac{\exp(-x_n)}{(1 + \exp(-x_n))^2}
                    \end{bmatrix}.
$$

(v)
>$$ f(x) = \log\left( \sum_{j=1}^n \exp(x_j) \right). $$

$$
    \nabla f(x) = \begin{bmatrix}
                    \frac{\exp(x_1)}{\sum_{j=1}^n \exp(x_j)} \\
                    \vdots \\
                    \frac{\exp(x_n)}{\sum_{j=1}^n \exp(x_j)}
                  \end{bmatrix},\ 
    \nabla^2 f(x) = (H_{ij})\ \mathrm{where}\ \ 
    H_{ij} = \begin{cases}
                \frac{\sum_{k\ne i}\exp(x_i + x_k)}{\left(\sum_{k=1}^n \exp(x_k)\right)^2} &\ \mathrm{if}\ \ i = j, \\
                \frac{-\exp(x_i + x_j)}{\left(\sum_{k=1}^n \exp(x_k)\right)^2} &\ \mathrm{if}\ \ i \ne j.
             \end{cases}
$$

# 3.2
>正定値対称行列 $A = (A_{ij}) \in \mathbb{R}^{n\times n}$, ベクトル $b \in \mathbb{R}^n$ に対し, 連立1次方程式
>$$ Ax = b \tag{3.32} $$
>の解と無制約最適化問題
>$$ \min f(x) = \frac{1}{2}x'Ax - b'x  \tag{3.33} $$
>の解が一致することを示せ. また, $\nabla^2 f(x) = A$ を示せ.

$A$ の対称性より, $f(x)$ の勾配およびヘッセ行列は,
$$
    \nabla f(x) = \frac{1}{2}(A + A')x - b = Ax - b,\ \nabla^2 f(x) = A.
$$
また $A$ は正定値行列なので, $f$ は狭義凸関数.
したがって $x^*$ が (3.33) の大域最適解であることの必要十分条件は,
$$
    \nabla f(x^*) = A x^* - b = 0
$$
である.

# 3.3
>3.1の(i)および(ii)の関数について, 初期点を適当に決め, 最急降下法とニュートン法の最初の2反復目までを実際に計算してみよ. 

In [6]:
def steepest_descent(x0, jac, alpha=0.2, k=2):
    x = x0.copy()
    for i in range(1, k+1):
        x = x - alpha * jac(x)
        print(f'Step {i}: x{i} = {x}')

In [7]:
def newton_method(x0, jac, hes, alpha=1.0, k=2):
    x = x0.copy()
    for i in range(1, k+1):
        d = np.linalg.solve(hes(x), -jac(x))
        x = x + alpha * d
        print(f'Step {i}: x{i} = {x}')

(i)

In [8]:
x0 = np.array([1.0, 1.0])
jac = lambda x: np.array([6*(x[0]**2)-2*x[0]*x[1], -(x[0]**2)+4*x[1]])
hes = lambda x: np.array([[12*x[0]-2*x[1], -2*x[0]], [-2*x[0], 4]])

print("最急降下法")
steepest_descent(x0, jac, alpha=0.2)

print("ニュートン法")
newton_method(x0, jac, hes, alpha=1.0)

最急降下法
Step 1: x1 = [0.2 0.4]
Step 2: x2 = [0.184 0.088]
ニュートン法
Step 1: x1 = [ 0.38888889 -0.05555556]
Step 2: x2 = [0.19911422 0.00090801]


(ii)

In [9]:
x0 = np.array([1.0, 1.0])
jac = lambda x: np.array([2*x[0]+x[1]-6, x[0]+(1/4)*(x[1]**3)+2*x[1]-2])
hes = lambda x: np.array([[2, 1], [1, (3/4)*(x[1]**2)+2]])

print("最急降下法")
steepest_descent(x0, jac, alpha=0.2)

print("ニュートン法")
newton_method(x0, jac, hes, alpha=1.0)

最急降下法
Step 1: x1 = [1.6  0.75]
Step 2: x2 = [2.01       0.50890625]
ニュートン法
Step 1: x1 = [ 3.11111111 -0.22222222]
Step 2: x2 = [ 3.32708612 -0.65417224]


# 3.4
>次の制約付き最適化問題に対するKKT条件を書け.

(i)
>$$
>\begin{align*}
>   \min \ \ &\frac{1}{2}\|Ax - b\|_2^2\\
>   \mathrm{s.t.}\ \ &x \ge 0.
>\end{align*}
>$$

$$
\begin{align*}
        &A'Ax -A'b - \lambda = 0,\\
        &\lambda_j x_j = 0, \ \lambda_j \ge 0, \ x_j \ge 0 \ \ \mathrm{for}\ \ j = 1,\dots,n.
\end{align*}
$$

(ii)
>$$
>\begin{align*}
>   \min \ \ &\sum_{j=1}^nx_j\log x_j\\
>   \mathrm{s.t.}\ \ &\sum_{j=1}^n x_j = 1,\\
>                    &x \ge 0.
>\end{align*}
>$$

$$
\begin{align*}
    &\log x_j + 1 + \mu - \lambda_j = 0 \ \ \mathrm{for}\ \ j = 1,\dots,n,\\
    &\sum_{j=1}^n x_j = 1,\\
    &\lambda_j x_j = 0,\ \lambda_j \ge 0,\ x_j \ge 0 \ \ \mathrm{for}\ \ j = 1,\dots,n.
\end{align*}
$$

# 3.5
>2変数の非線形計画問題の例で, 局所最適解において1次独立制約想定が成り立たないものを作り, その局所最適解でKKT条件を満たすラグランジュ乗数が存在するかを調べよ.

次の制約付き最適化問題を考える：
$$
\begin{align*}
    \min \ \ &-x_1\\
    \mathrm{s.t.}\ \ & x_2 - (1 - x_1)^3 \le 0,\\
    &x_1, x_2 \ge 0.
\end{align*}
$$
この問題の最適解 $x^*$ は $(x_1^*, x_2^*) = (1, 0)$ である（$\because x_1 \ge 1$ の領域において制約を満たす点は $x^* = (1, 0)$ のみであるため）.

$x^*$ における有効制約は $g_1(x) := x_2 - (1 - x_1)^3 \le 0$ と $g_2(x) := -x_2 \le 0$ だが, 
$$
    \nabla g_1(x)|_{x = x^*} = \left.\begin{bmatrix} 3(1-x_1)^2 \\ 1\end{bmatrix}\right|_{x=x^*}
                             = \begin{bmatrix} 0\\ 1\end{bmatrix}, \ \ 
    \nabla g_2(x)|_{x = x^*} = \begin{bmatrix}0 \\ -1\end{bmatrix}
$$
より, 最適解 $x^*$ は一次独立制約想定を満たさない.

KKT条件は, ラグランジュ乗数を $\lambda_0, \lambda_1, \lambda_2$ として
$$
\begin{align*}
    &-1 + 3\lambda_0(1-x_1)^2 - \lambda_1 = 0\\
    &\lambda_0 - \lambda_2 = 0\\
    &\lambda_0 (x_2 - (1 - x_1)^3) = \lambda_1 x_1 = \lambda_2 x_2 = 0\\
    &x_2 - (1 - x_1)^3 \le 0\\
    &\lambda_0, \lambda_1, \lambda_2, x_1, x_2 \ge 0.
\end{align*}
$$
となるが, $x^* = (1, 0)$ においては1つ目の条件から $\lambda_1 = -1$ となってしまうので, $x^*$ においてKKT条件を満たすラグランジュ乗数は存在しない.

# 3.6
>$\ell_1$ ノルム正則化付き最小2乗法
>$$ \min_{x}\ \  \frac{1}{2}\left\| \sum_{j=1}^n x_j a_j - b \right\|_2^2 + \gamma \sum_{j=1}^n |x_j| $$
>に対する座標降下法の更新式を求めよ.

目的関数を $f(x)$ とおき, これを $x_l$ について整理すると,
$$
\begin{align*}
    f(x) &= \frac{1}{2}\left\| x_l a_l + \sum_{j\ne l}x_j a_j - b \right\|_2^2 + \gamma |x_l| + \gamma \sum_{j\ne l}|x_j| \\
    &= \frac{1}{2}\left(\|a_l\|_2^2 x_l^2 + 2 \left(\sum_{j\ne l}x_j a_j - b\right)'a_l x_l \right) + \gamma |x_l| + C\\
    &= \begin{cases}
            \frac{1}{2}\left(\|a_l\|_2^2 x_l^2 + 2 \left(\sum_{j\ne l}x_j a_j - b\right)'a_l x_l \right) + \gamma x_l + C & \ \mathrm{if}\ \ x_l \ge 0\\
            \frac{1}{2}\left(\|a_l\|_2^2 x_l^2 + 2 \left(\sum_{j\ne l}x_j a_j - b\right)'a_l x_l \right) - \gamma x_l + C & \ \mathrm{if}\ \ x_l < 0
       \end{cases}
\end{align*}
$$
となる（ただし $C$ は $x_l$ と無関係な項）.
よって $x_{-l}$ を固定したときに $f(x_l, x_{-l})$ を最小化する点 $x_l$ の候補は, 
- $p = \frac{1}{\|a_l\|_2^2} \left( \left( b - \sum_{j \ne l}a_jx_j \right)'a_l - \gamma \right) $ $ \ \cdots \ $ (  $x_l > 0$ 領域の放物線の頂点)
- $q = 0$ $ \ \cdots \ $ ( 領域の境界 )
- $r = \frac{1}{\|a_l\|_2^2} \left( \left( b - \sum_{j \ne l}a_jx_j \right)'a_l + \gamma \right) $ $ \ \cdots \ $ (  $x_l < 0$ 領域の放物線の頂点)

に絞られる.

(Case 1): $\left( b - \sum_{j \ne l}a_jx_j \right)'a_l > \gamma$ の場合.
- $0 = q < p < r$ となるので, $f$ が最小になるのは $x_l = p$ のとき.

(Case 2): $\left( b - \sum_{j \ne l}a_jx_j \right)'a_l < -\gamma$ の場合.
- $p < r < q = 0$ となるので, $f$ が最小になるのは $x_l = r$ のとき.

(Case 3): $-\gamma \le \left( b - \sum_{j \ne l}a_jx_j \right)'a_l \le \gamma$ の場合.
- $p < q = 0 < r$ となるので, $f$ が最小になるのは $x_l = q$ のとき.

よって座標降下法の更新式は, 
$$
    x_l \leftarrow \frac{1}{\|a_l\|_2^2} \psi\left( \left( b - \sum_{j \ne l}a_jx_j \right)'a_l \right)
$$
と書ける, ただし $\psi$ は
$$
    \psi(s) = \begin{cases}
                  s - \gamma & \ \ \mathrm{if}\ \ s > \gamma,\\
                  0 & \ \ \mathrm{if}\ \ |s| \le \gamma,\\
                  s + \gamma & \ \ \mathrm{if}\ \ s < -\gamma
              \end{cases}
$$
なる関数.